In [4]:
pip install clarifai_grpc

Note: you may need to restart the kernel to use updated packages.


In [5]:
########################################################################################
# In this section, we set the user authentication, app ID, model details, and the URL
# of the image we want as an input. Change these strings to run your own example.
#######################################################################################

USER_ID = 'uy9ljv35e76h'
# Your PAT (Personal Access Token) can be found in the portal under Authentification
PAT = '05d063165ded43c88d60773f22118cd1'
APP_ID = 'fe1e29f5552d430d8365fc782a446809'
# Change this to whatever image URL you want to process
IMAGE_URL = 'https://a0.muscache.com/im/pictures/user/c22d19f5-bf5f-48b3-a6b7-7f8abd5c20e4.jpg'
# This is optional. You can specify a model version or the empty string for the default
MODEL_VERSION_ID = ''

############################################################################
# YOU DO NOT NEED TO CHANGE ANYTHING BELOW THIS LINE TO RUN THIS EXAMPLE
############################################################################
from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
from clarifai_grpc.grpc.api.status import status_code_pb2

channel = ClarifaiChannel.get_grpc_channel()
stub = service_pb2_grpc.V2Stub(channel)

metadata = (('authorization', 'Key ' + PAT),)

userDataObject = resources_pb2.UserAppIDSet(user_id=USER_ID, app_id=APP_ID)


In [ ]:
############################################################################
# This model will be used for predicting ethnicity or race of users
############################################################################
MODEL_ID = 'ethnicity-demographics-recognition'  ##This need to change

post_model_outputs_response = stub.PostModelOutputs(
    service_pb2.PostModelOutputsRequest(
        user_app_id=userDataObject,  # The userDataObject is created in the overview and is required when using a PAT
        model_id=MODEL_ID,
        version_id=MODEL_VERSION_ID,  # This is optional. Defaults to the latest model version
        inputs=[
            resources_pb2.Input(
                data=resources_pb2.Data(
                    image=resources_pb2.Image(
                        url=IMAGE_URL
                    )
                )
            )
        ]
    ),
    metadata=metadata
)
if post_model_outputs_response.status.code != status_code_pb2.SUCCESS:
    print(post_model_outputs_response.status)
    raise Exception("Post model outputs failed, status: " + post_model_outputs_response.status.description)

# Since we have one input, one output will exist here
output = post_model_outputs_response.outputs[0]

# Uncomment this line to print the full Response JSON
print(output.data)

print("Predicted concepts:")
import json
import operator

predictedconcepts = {}
for concept in output.data.concepts:
  predictedconcepts[concept.value] = str(concept.name)

print(predictedconcepts)
print(predictedconcepts.get(max(predictedconcepts)))

In [ ]:
#importing deepface library and DeepFace
import mysql.connector
from datetime import datetime


print('start at: ' + str(datetime.now()))
mydb = mysql.connector.connect(
            host = "localhost",
            user = "root",
            password = "mysql@123",
            database = "airbnb2022",
            auth_plugin='mysql_native_password'
        )

cursor = mydb.cursor()
query = "SELECT * FROM airbnb2022.airbnb_listing_users_usa_ny_predicted_clarifai where length(gender_tool) > 1;"
cursor.execute(query)
 
myresult = cursor.fetchall()

print(myresult)


In [56]:
def getPredictionsForUrl(MODEL_ID, url):
############################################################################
# This model will be used for predicting ethnicity and gender based on model id passed from main method
############################################################################

    post_model_outputs_response = stub.PostModelOutputs(
        service_pb2.PostModelOutputsRequest(
            user_app_id=userDataObject,  # The userDataObject is created in the overview and is required when using a PAT
            model_id=MODEL_ID,
            version_id=MODEL_VERSION_ID,  # This is optional. Defaults to the latest model version
            inputs=[
                resources_pb2.Input(
                    data=resources_pb2.Data(
                        image=resources_pb2.Image(
                            url=url
                        )
                    )
                )
            ]
        ),
        metadata=metadata
    )
    if post_model_outputs_response.status.code != status_code_pb2.SUCCESS:
        print(post_model_outputs_response.status)
        #raise Exception("Post model outputs failed, status: " + post_model_outputs_response.status.description)
        return ""

    # Since we have one input, one output will exist here
    output = post_model_outputs_response.outputs[0]

    # Uncomment this line to print the full Response JSON
    print(output.data)

    print("Predicted concepts:")
    import json
    import operator

    predictedconcepts = {}
    for concept in output.data.concepts:
        predictedconcepts[concept.value] = str(concept.name)

    print(predictedconcepts)
    return str(predictedconcepts.get(max(predictedconcepts)))

In [ ]:
############################################################################
# This section call the model prediction for each row in the resulset
############################################################################
import pandas as pd
count = 0
df = pd.DataFrame(myresult)
print(df.columns.values)
for x in myresult:
    print(x)
    url = x[3]

    #For Gender Prediction:
    GENDER_MODEL_ID = 'gender-demographics-recognition' 
    df.loc[count,[10]] = getPredictionsForUrl(GENDER_MODEL_ID, url);
    
    #For ethnicity prediction
    ETHNICITY_MODEL_ID = 'ethnicity-demographics-recognition' 
    df.loc[count,[12]] = getPredictionsForUrl(ETHNICITY_MODEL_ID, url);
    
    count+=1
   

In [13]:
############################################################################
# Set up data frame to calculate confusion matrix
############################################################################
import pandas as pd
count = 0
df = pd.DataFrame(myresult)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,1,Nathan,None,https://a0.muscache.com/im/users/1/profile_pic/1266266786/original.jpg,man,None,white,None,None,None,Feminine,East Asian
1,1,100000864,Tove,None,https://a0.muscache.com/im/pictures/user/64885f42-9a26-4682-80f9-3733bd719e2b.jpg,woman,None,white,None,None,None,Masculine,Southeast Asian
2,2,10000166,Jennifer,None,https://a0.muscache.com/im/pictures/user/c5a9712b-3865-4f7d-845b-b57e7b06f855.jpg,woman,None,white,None,None,None,Masculine,White
3,3,100002843,Hernán,None,https://a0.muscache.com/im/pictures/user/4c5f3feb-b214-46d0-955d-53f187687a97.jpg,man,None,white,None,None,None,Masculine,Black
4,4,100003253,Anneth,None,https://a0.muscache.com/im/pictures/user/be98b685-6601-4ff1-b43d-bcbb5fa68bed.jpg,woman,None,asian,None,None,None,Feminine,East Asian


In [20]:
clarifaiResultset = pd.DataFrame(df);
print(clarifaiResultset.columns.values)
clarifaiResultset[12].replace({'Southeast Asian':'asian'},inplace=True)
clarifaiResultset[12].replace({'East Asian':'asian'},inplace=True)
clarifaiResultset[11].replace({'Masculine':'man'},inplace=True)
clarifaiResultset[11].replace({'Feminine':'woman'},inplace=True)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12]


In [ ]:
clarifaiResultset.drop(clarifaiResultset.columns[[0, 1, 2, 3,4, 6, 8,9, 10]], axis=1, inplace=True)


In [24]:

clarifaiResultset[7].replace({'asia':'asian'},inplace=True)
clarifaiResultset

,0,1,2,3,...,9,10,11,12
0,0,1,Nathan,None,...,None,None,woman,asian
1,1,100000864,Tove,None,...,None,None,man,asian
2,2,10000166,Jennifer,None,...,None,None,man,White
3,3,100002843,Hernán,None,...,None,None,man,Black
4,4,100003253,Anneth,None,...,None,None,woman,asian
5,5,100004888,Sydney,None,...,None,None,woman,Black
6,6,100004951,Charles,None,...,None,None,man,Latino_Hispanic
7,7,100008667,John,None,...,None,None,man,Latino_Hispanic
8,8,10000871,Renan,None,...,None,None,man,White
9,9,100011780,Joshua,None,...,None,None,man,asian


In [26]:
#Clarifai - Confusion matrix for gender prediction
confusion_matrix_gender = pd.crosstab(clarifaiResultset[5].str.lower(), clarifaiResultset[11].str.lower(), rownames=['Actual'], colnames=['Predicted'])
confusion_matrix_gender

Predicted,man,woman
Actual,,
man,78,14
woman,31,37


In [27]:
#Clarifai - Confusion matrix for ethnicity prediction
confusion_matrix_race = pd.crosstab(clarifaiResultset[7].str.lower(), clarifaiResultset[12].str.lower(), rownames=['Actual'], colnames=['Predicted'])
pd.set_option('display.max_columns', confusion_matrix_race.shape[0]+1)
confusion_matrix_race

Predicted,asian,black,indian,latino_hispanic,middle eastern,white
Actual,,,,,,
asian,14,3,1,3,0,4
black,4,7,2,0,0,1
hispanic,1,3,0,8,0,2
indian,0,0,2,0,0,0
middle eastern,4,1,0,0,0,0
white,31,14,2,3,7,42
